# Dependencies

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from collections import Counter

In [ ]:
def plt_imshow(title, image):
    # convert the image frame BGR to RGB color space and display it
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(title)
    plt.grid(False)
    plt.show()

In [ ]:
num2class = {"0.0" : "buffalo", "1.0" : "elephant", "2.0" : "rhino", "3.0" : "zebra"}

# Bounding boxes

In [ ]:
# box : (centerX, centerY, width, height)
def convertToAbsoluteValues(size, box):
    
    xIn = round(((2 * float(box[0]) - float(box[2])) * size[0] / 2))
    yIn = round(((2 * float(box[1]) - float(box[3])) * size[1] / 2))
    xEnd = xIn + round(float(box[2]) * size[0])
    yEnd = yIn + round(float(box[3]) * size[1])
    
    if xIn < 0:
        xIn = 0
    if yIn < 0:
        yIn = 0
    if xEnd >= size[0]:
        xEnd = size[0] - 1
    if yEnd >= size[1]:
        yEnd = size[1] - 1
    return (xIn, yIn, xEnd, yEnd)

def convertToRelativeValues(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    cx = (box[1] + box[0]) / 2.0
    cy = (box[3] + box[2]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = cx * dw
    y = cy * dh
    w = w * dw
    h = h * dh
    # x,y => (bounding_box_center)/width_of_the_image
    # w => bounding_box_width / width_of_the_image
    # h => bounding_box_height / height_of_the_image
    return (x, y, w, h)

In [ ]:
import cv2
imgfilepath = os.path.join("image", "022" + ".jpg")
print(imgfilepath)
img = cv.imread(imgfilepath)
plt_imshow("", img)
h, w, _ = img.shape
print(h,w)
label, conf, rx1, ry1, rx2, ry2 = (3, 1, 0.572656, 0.523958, 0.698438, 0.589583)
cood = convertToAbsoluteValues((w,h), (rx1, ry1, rx2, ry2 ))
print(cood)

clone = img.copy()
cv2.rectangle(clone, (cood[0], cood[1]), (cood[2], cood[3]), (255, 0, 0), 2)
plt_imshow("", clone)



In [ ]:
list(range(10))

In [ ]:
labelPath = "label"
os.listdir(labelPath)
os.listdir("label/detection")

In [ ]:
a = [1,2,3,4,5]

b = list(map(float, a))
print(b)

In [ ]:
for data in range(5):
    print(data)

In [ ]:
labelPath = "label"
imagePath = "image"
detections, groundtruths, classes = [], [], []
for boxtype in os.listdir(labelPath):
#     print(boxtype)

    boxtypeDir = os.path.join(labelPath,boxtype)
#     print(boxtypeDir)
   

    for labelfile in os.listdir(boxtypeDir):
#         print(labelfile)
#         print(os.path.splitext(labelfile))
        
        filename = os.path.splitext(labelfile)[0]
#         print(filename)
#         print(os.path.join(boxtypeDir, labelfile))
        
        with open(os.path.join(boxtypeDir, labelfile)) as f:
            labelinfos = f.readlines()
#         print(labelinfos)
        

        imgfilepath = os.path.join(imagePath, filename + ".jpg")
#         print(imgfilepath)
        img = cv.imread(imgfilepath)
        h, w, _ = img.shape
#         print(w,h)
        
        for labelinfo in labelinfos:
#             print(labelinfo, type(labelinfo))
            
#             print(labelinfo.strip().split())
            
#             label, conf, rx1, ry1, rx2, ry2  = map(float, labelinfo.strip().split())
#             print(label, conf, rx1, ry1, rx2, ry2 )
            
            label, conf, rx1, ry1, rx2, ry2 = map(float, labelinfo.strip().split())
            x1, y1, x2, y2 = convertToAbsoluteValues((w, h), (rx1, ry1, rx2, ry2))
            boxinfo = [filename, label, conf, (x1, y1, x2, y2)]
#             print(boxinfo)
            
                
            if label not in classes:
                classes.append(label)
#             print(classes)
            
                
            if boxtype == "detection":
                detections.append(boxinfo)
            else:
                groundtruths.append(boxinfo)

print(classes)
print(detections)
print(groundtruths)
        
    

In [ ]:
def boundingBoxes(labelPath, imagePath):
    
    detections, groundtruths, classes = [], [], []
    
    for boxtype in os.listdir(labelPath):

        boxtypeDir = os.path.join(labelPath,boxtype)

        for labelfile in os.listdir(boxtypeDir):
            filename = os.path.splitext(labelfile)[0]
            with open(os.path.join(boxtypeDir, labelfile)) as f:
                labelinfos = f.readlines()

            imgfilepath = os.path.join(imagePath, filename + ".jpg")
            img = cv.imread(imgfilepath)
            h, w, _ = img.shape

            for labelinfo in labelinfos:
                label, conf, rx1, ry1, rx2, ry2 = map(float, labelinfo.strip().split())
                x1, y1, x2, y2 = convertToAbsoluteValues((w, h), (rx1, ry1, rx2, ry2))
                boxinfo = [filename, label, conf, (x1, y1, x2, y2)]
                
                if label not in classes:
                    classes.append(label)
                
                if boxtype == "detection":
                    detections.append(boxinfo)
                else:
                    groundtruths.append(boxinfo)
                    
    classes = sorted(classes)
                
    return detections, groundtruths, classes

detections, groundtruths, classes = boundingBoxes("label", "image")
print(detections)
print(groundtruths)
print(classes)

# Plot Bounding boxes

In [ ]:
# os.mkdir("boxed_images/detection")
os.mkdir("boxed_images/groundtruth")

In [ ]:
def boxPlot(boxlist, imagePath, savePath):
    labelfiles = sorted(list(set([filename for filename, _, _, _ in boxlist])))
    
    for labelfile in labelfiles:
    
        rectinfos = []
        imgfilePath = os.path.join(imagePath, labelfile + ".jpg")
        img = cv.imread(imgfilePath)

        for filename, _, conf, (x1, y1, x2, y2) in boxlist:
            if labelfile == filename:
                rectinfos.append((x1, y1, x2, y2, conf))
                
        for x1, y1, x2, y2, conf in rectinfos:
            
            if conf == 1.0:
                rectcolor = (0, 255, 0)
            else:
                rectcolor = (0, 0, 255)
                
            cv.rectangle(img, (x1, y1), (x2, y2), rectcolor, 4)
        cv.imwrite(f"{savePath}/{labelfile}.jpg", img)

        img = mpimg.imread(f"{savePath}/{labelfile}.jpg")
        plt.axis("off")
        plt.title(labelfile)
        plt.imshow(img)
        plt.show()
        

# boxPlot(detections, "image", savePath="boxed_images/detection")
# boxPlot(groundtruths, "image", savePath="boxed_images/groundtruth")
boxPlot(detections + groundtruths, "image", savePath="boxed_images/both")

# IoU(Intersection over Union)

In [ ]:
def getArea(box):
    return (box[2] - box[0] + 1) * (box[3] - box[1] + 1)


def getUnionAreas(boxA, boxB, interArea=None):
    area_A = getArea(boxA)
    area_B = getArea(boxB)
    
    if interArea is None:
        interArea = getIntersectionArea(boxA, boxB)
        
    return float(area_A + area_B - interArea)

def getIntersectionArea(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # intersection area
    return (xB - xA + 1) * (yB - yA + 1)

# boxA = (Ax1,Ay1,Ax2,Ay2)
# boxB = (Bx1,By1,Bx2,By2)
def boxesIntersect(boxA, boxB):
    if boxA[0] > boxB[2]:
        return False  # boxA is right of boxB
    if boxB[0] > boxA[2]:
        return False  # boxA is left of boxB
    if boxA[3] < boxB[1]:
        return False  # boxA is above boxB
    if boxA[1] > boxB[3]:
        return False  # boxA is below boxB
    return True

def iou(boxA, boxB):
    # if boxes dont intersect
    if boxesIntersect(boxA, boxB) is False:
        return 0
    interArea = getIntersectionArea(boxA, boxB)
    union = getUnionAreas(boxA, boxB, interArea=interArea)
    
    # intersection over union
    result = interArea / union
    assert result >= 0
    return result

In [ ]:
boxA = detections[14][-1]
boxB = groundtruths[14][-1]

print(f"boxA coordinates : {(boxA)}")
print(f"boxA area : {getArea(boxA)}")
print(f"boxB coordinates : {(boxB)}")
print(f"boxB area : {getArea(boxB)}")

print(f"Union area of boxA and boxB : {getUnionAreas(boxA, boxB)}")

print(f"Does boxes Intersect? : {boxesIntersect(boxA, boxB)}")

print(f"Intersection area of boxA and boxB : {getIntersectionArea(boxA, boxB)}")

print(f"IoU of boxA and boxB : {iou(boxA, boxB)}")

# AP(Average Precision)

In [ ]:
def calculateAveragePrecision(rec, prec):
    
    mrec = [0] + [e for e in rec] + [1]
    print("mrec=", mrec)
    mpre = [0] + [e for e in prec] + [0]
    print("mpre=", mpre)

    for i in range(len(mpre)-1, 0, -1):
        mpre[i-1] = max(mpre[i-1], mpre[i])
        
    print("mpre=", mpre)

    ii = []

    for i in range(len(mrec)-1):
        if mrec[1:][i] != mrec[0:-1][i]:
            ii.append(i+1)
            
    print("ii=", ii)

    ap = 0
    for i in ii:
        ap = ap + np.sum((mrec[i] - mrec[i-1]) * mpre[i])
        print("ap=", ap)
    
    return [ap, mpre[0:len(mpre)-1], mrec[0:len(mpre)-1], ii]

In [ ]:
def ElevenPointInterpolatedAP(rec, prec):

    mrec = [e for e in rec]
    mpre = [e for e in prec]

    recallValues = np.linspace(0, 1, 11)
    recallValues = list(recallValues[::-1])
    rhoInterp, recallValid = [], []

    for r in recallValues:
        argGreaterRecalls = np.argwhere(mrec[:] >= r)
        pmax = 0

        if argGreaterRecalls.size != 0:
            pmax = max(mpre[argGreaterRecalls.min():])

        recallValid.append(r)
        rhoInterp.append(pmax)

    ap = sum(rhoInterp) / 11

    return [ap, rhoInterp, recallValues, None]

In [ ]:
print(detections)
print(groundtruths)
print(classes)

In [ ]:
a = np.array([1,2,3,4,5,6])
b = np.cumsum(a)
print(b)

In [ ]:
def AP(detections, groundtruths, classes, IOUThreshold = 0.3, method = 'AP'):
    
    result = []
    
    for c in classes:

        dects = [d for d in detections if d[1] == c]
        gts = [g for g in groundtruths if g[1] == c]
        
#         print('dects', dects)
#         print('gts', gts)
 

        
        npos = len(gts)
#         print(npos)
        
        
        

#         for conf in dects:
#             print(conf, conf[2])
            
        dects = sorted(dects, key = lambda conf : conf[2], reverse=True)
#         print(dects)
        
        
        TP = np.zeros(len(dects))
        FP = np.zeros(len(dects))
        
        
        det = Counter(cc[0] for cc in gts)
#         print(det)
        
        
        # 각 이미지별 ground truth box의 수
        # {99 : 2, 380 : 4, ....}
        # {99 : [0, 0], 380 : [0, 0, 0, 0], ...}
        for key, val in det.items():
            det[key] = np.zeros(val)
            
#         print(det)
#         for key, val in det.items():
#             print(key, val)
        
        for d in range(len(dects)):
            gt = [gt for gt in gts if gt[0] == dects[d][0]]
#             print(gt)
            

            iouMax = 0

            for j in range(len(gt)):
                iou1 = iou(dects[d][3], gt[j][3])
#                 print(iou1)
#                 break
                if iou1 > iouMax:
                    iouMax = iou1
                    jmax = j
                    
            if iouMax >= IOUThreshold:
                if det[dects[d][0]][jmax] == 0:
                    TP[d] = 1
                    det[dects[d][0]][jmax] = 1
                else:
                    FP[d] = 1
            else:
                FP[d] = 1
                
        
                
#         print(TP)
#         print(FP)
        
#         for key, val in det.items():
#             print(key, val)
            
        
        acc_FP = np.cumsum(FP)
#         print(acc_FP)
      
        acc_TP = np.cumsum(TP)
#         print(acc_TP)
        
        rec = acc_TP / npos
#         print(rec)
        prec = np.divide(acc_TP, (acc_FP + acc_TP))
#         print(prec)
        

        if method == "AP":
            [ap, mpre, mrec, ii] = calculateAveragePrecision(rec, prec)
        else:
            [ap, mpre, mrec, _] = ElevenPointInterpolatedAP(rec, prec)
        
        print([ap, mpre, mrec, ii])
#         break

        r = {
            'class' : c,
            'precision' : prec,
            'recall' : rec,
            'AP' : ap,
            'interpolated precision' : mpre,
            'interpolated recall' : mrec,
            'total positives' : npos,
            'total TP' : np.sum(TP),
            'total FP' : np.sum(FP)
        }

        result.append(r)

    return result

result = AP(detections, groundtruths, classes)
print("result=", result)

# mAP(mean Average Precision)

In [ ]:
def mAP(result):
    ap = 0
    for r in result:
        ap += r['AP']
    mAP = ap / len(result)
    
    return mAP

In [ ]:
for r in result:
    print("{:^8} AP : {}".format(num2class[str(r['class'])], r['AP']))
print("---------------------------")
print(f"mAP : {mAP(result)}")